<a href="https://colab.research.google.com/github/Anastasiamay/Study_projects/blob/main/credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Исследование надежности заемщиков


## Подгрузка и изучение датасета

In [ ]:
import pandas as pd
!gdown --id 1UhjnMZE4Uw_pnTs3dxOOMDPRs863eP3h

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1UhjnMZE4Uw_pnTs3dxOOMDPRs863eP3h
To: /content/data.csv
100% 3.32M/3.32M [00:00<00:00, 194MB/s]


In [ ]:
!pip install -q xlrd
data = pd.read_csv('data.csv')
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Были обнаружены пропущенные значения в трудовом стаже и заработной плате. Заменим медианными значениями в соответствие с типом занятости.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

### Обработка аномальных значений

In [ ]:
data['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

В трудовом стаже имеются отрицательные значение. Уберем знак "-".

In [ ]:
data['days_employed'] = data['days_employed'].abs()

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Но этот столбец не понадобится для исследования.

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Также аномальные значения обнаружены в количестве детей. В дальнейшем анализе не будем их учитывать.

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [ ]:
data['total_income'].unique()

array([253875.6394526 , 112080.01410244, 145885.95229686, ...,
        89672.56115303, 244093.05050043,  82047.41889948])

Для удобства преобразуем ежемесячный доход к целочисленному значению.

In [ ]:
data['total_income'] = data['total_income'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Обработка дубликатов

In [ ]:
data['education'].value_counts()

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Образование записано разным регистром, приведем к одному регистру. Удалим дубликаты.

In [ ]:
data['education'] = data['education'].str.lower()

In [ ]:
data.duplicated().sum()

54

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

Для категоризации заёмщиков по доходу создадим функцию

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

NameError: ignored

In [ ]:
data['purpose'].unique()

Для категоризации по цели кредита можно выделить 4 основные цели: операции с автомобилем, операции с недвижимостью, проведение свадьбы, получение образования.

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

# Исследоваение данных

## Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
pivot_table_children = data.pivot_table(index = 'children', columns = 'debt', values = 'gender', aggfunc = 'count')
pivot_table_children['Процент задолженностей'] = \
round(pivot_table_children[1] / (pivot_table_children[0] + pivot_table_children[1]) * 100, 2)
pivot_table_children

debt,0,1,Процент задолженностей
children,,,
0,13028.0,1063.0,7.54
1,4364.0,444.0,9.23
2,1858.0,194.0,9.45
3,303.0,27.0,8.18
4,37.0,4.0,9.76
5,9.0,NaN,NaN


**Вывод:** В выборку попало слишком мало заемщиков с пятью и четырьмя детьми. Попробуем категоризировать по типу семьи в зависимости от количества детей. Таким образом, если количество детей = 0, назовем заёмщика бездетным, 1-2 ребенка - малодетным, от 3 и более - многодетным.

In [ ]:
def count_of_children(count):
    try:
        if count == 0:
            return 'бездетные'
        elif 1 <= count <= 2:
            return 'малодетные'
        elif count >= 3:
            return 'многодетные'
    except:
        pass    

In [ ]:
data['family_children_status'] = data['children'].apply(count_of_children)

In [ ]:
pivot_table_children = data.pivot_table(index = 'family_children_status', columns = 'debt', values = 'gender', aggfunc = 'count')
pivot_table_children['Процент задолженностей'] = \
round(pivot_table_children[1] / (pivot_table_children[0] + pivot_table_children[1]) * 100, 2)
pivot_table_children.sort_values(by = 'Процент задолженностей')

debt,0,1,Процент задолженностей
family_children_status,,,
бездетные,13028,1063,7.54
многодетные,349,31,8.16
малодетные,6222,638,9.30


**Вывод:** Наиболее часто кредиты возвращают в срок бездетные заёмщики, что логично, ведь им не надо выделять бюджет на детей. Заёмщики с 1-2 детьми возвращают кредиты значительно хуже. Но многодетные возвращают кредиты лучше, малодетных, можно предположить, что выборка по многодетным слишком маленькая. В целом, после категоризации результаты не поменялись и наблюдается тенденция увелечения процента задолженностей с увелечением количества детей.

##Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
pivot_table_family = data.pivot_table(index = 'family_status', columns = 'debt', values = 'gender', aggfunc = 'count')
pivot_table_family['Процент задолженностей'] = \
round(pivot_table_family[1] / (pivot_table_family[0] + pivot_table_family[1]) * 100, 2)
pivot_table_family.sort_values(by = 'Процент задолженностей')

debt,0,1,Процент задолженностей
family_status,,,
вдовец / вдова,888,63,6.62
в разводе,1105,84,7.06
женат / замужем,11334,927,7.56
гражданский брак,3749,385,9.31
Не женат / не замужем,2523,273,9.76


**Вывод:** Чаще всего кредиты возвращают в срок вдовцы/вдовы и заёмщики в разводе. Чуть выше процент задолженности у женатых/замужних, но все-таки ближе к предыдущим категориям. А вот заёмщики в гражданском браке и не женатые/не замужем гораздо чаще имеют задолженности по кредитам. 

Можно предположить, что вдовцы/вдовы и разведенные - люди, уже более ответственные, и понимают, что им не на кого полагаться. 

Заёмщики в браке более ответственно подходят к планированию бюджета, что также сказывается на погашении кредита в срок. 

Заёмщики в гражданском браке и не состоящие в браке имеют примерно равный процент задолженности, что обуславливается схожим статусом, ведь гражданский брак никак не подкреплен юридически и такие заёмщики могут относиться к категории не состоящих в браке; обе категории заёмщиков менее ответственны в распределении бюджета.

##Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
pivot_table_income = data.pivot_table(index = 'total_income_category', columns = 'debt', values = 'gender', aggfunc = 'count')
pivot_table_income['Процент задолженностей'] = \
round(pivot_table_income[1] / (pivot_table_income[0] + pivot_table_income[1]) * 100, 2)
pivot_table_income.sort_values(by = 'Процент задолженностей', ascending=False)

debt,0,1,Процент задолженностей
total_income_category,,,
E,20,2,9.09
C,14568,1353,8.50
A,23,2,8.00
B,4660,354,7.06
D,328,21,6.02


**Вывод:** Видим, что группы A, E и D представляют маленькие выборки. У нас остается две группы, чтобы оценить зависимость, так можно сказать, что процент задолженностей ниже у заёмщиков с большим ежемесячным доходом, поскольку такие люди реже имеют проблемы с деньгами.

##Как разные цели кредита влияют на его возврат в срок?

In [ ]:
pivot_table_purpose = data.pivot_table(index = 'purpose_category', columns = 'debt', values = 'gender', aggfunc = 'count')
pivot_table_purpose['Процент задолженностей'] = \
round(pivot_table_purpose[1] / (pivot_table_purpose[0] + pivot_table_purpose[1]) * 100, 2)
pivot_table_purpose.sort_values(by = 'Процент задолженностей', ascending=False)

debt,0,1,Процент задолженностей
purpose_category,,,
операции с автомобилем,3879,400,9.35
получение образования,3619,369,9.25
проведение свадьбы,2130,183,7.91
операции с недвижимостью,9971,780,7.26


**Вывод:** Меньше всего задолженностей по кредитам, связанных с недвижимостью, скорее всего потому, что для большинства людей кредит на недвижимость является самым большим по стоимости, поэтому относятся к нему с большей ответственностью. Чуть хуже возвращают кредит, взятый на свадьбу. Могу предположить, что если люди берут кредит на свадьбу, то скорее всего часть этого кредита они погашают подаренными деньгами. Значительно хуже и почти одинаково возвращают кредиты, взятые на образование и автомобиль. Возможно, у заёмщика обучение занимает много времени, и доход тем самым падает, поэтому происходят задолженности. При автокредите, скорее всего, появляются дополнительные расходы на автомобиль, что объясняет задолженности.

#Общий вывод

По результатам анализа выявили, что все рассмотренные факторы, а именно: количество детей, семейное положение, уровень дохода и цели кредита влияют на возврат кредита в срок. Подведем итоговые выводы:
* Лучше всего кредиты возвращают в срок заёмщики без детей, а с увеличением количества детей кредиты возвращают хуже. 
* Наименьший процент задолженностей у вдовцов/вдов, а хуже всего возвращают кредиты не женатые/не замужние заёмщики.
* В данных были пропуски по ежемесячному доходу, которые были необходимы для определения зависимости между уровнем дохода и возвратом кредита в срок. Для избежания потерь данных заменили пропуски медианным значением в соответствии с типом занятости. В целом можно отметить, что с увеличением дохода процент задолженностей уменьшается.
* Наиболее часто возвращают в срок кредиты, связанным с недвижимостью. И, почти одинаково плохо, автокредиты и кредиты на образование. 